# 데모 데이터 생성

In [42]:
import os
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
import json
import re
import sys
import time

In [43]:
msn_train= pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/msn_train.csv')

In [44]:
demo_tr = pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/MINDdemo_train/news.tsv', delimiter='\t', header=None)
print(demo_tr.shape)
demo_tr.head()

(26740, 8)


,0,1,2,3,4,5,6,7
0,N3112,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://www.msn.com/en-us/lifestyle/lifestyler...,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
2,N12103,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://www.msn.com/en-us/health/voices/i-was-...,[],"[{""Label"": ""National Basketball Association"", ..."
3,N20460,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://www.msn.com/en-us/health/medical/how-t...,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
4,N5409,weather,weathertopstories,It's been Orlando's hottest October ever so fa...,There won't be a chill down to your bones this...,https://www.msn.com/en-us/weather/weathertopst...,"[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W...","[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W..."


In [45]:
demo_tr_nid = demo_tr.iloc[:,0]
print(len(demo_tr_nid))

demo_trains = msn_train.loc[msn_train.iloc[:,6].isin(demo_tr_nid)]
print(demo_trains.shape)
demo_trains.head() 

26740
(21007, 13)


,5,nids,Title,Date,Image,Content,0,1,2,3,4,6,7
2,https://assets.msn.com/labs/mind/AAEKnO1.html,AAEKnO1,Check Houston traffic map for current road con...,2019-11-26T12:55:03Z,https://img-s-msn-com.akamaized.net/tenant/amp...,Get real-time traffic data from Houston's road...,N5771,health,cardio,Check Houston traffic map for current road con...,NaN,"[{""Label"": ""Houston"", ""Type"": ""G"", ""WikidataId...",[]
4,https://assets.msn.com/labs/mind/AAGBR44.html,AAGBR44,This Guy Altered His Diet and Training to Drop...,2019-08-31T11:37:11Z,https://img-s-msn-com.akamaized.net/tenant/amp...,Brandon Reid always felt like âthe fat kid.â...,N17957,health,weightloss,This Guy Altered His Diet and Training to Drop...,"Take Brandon Reid's advice: ""Don't worry what ...",[],[]
14,https://assets.msn.com/labs/mind/AABrOCk.html,AABrOCk,The States Where Dogs Put the Biggest Bite on ...,2019-06-18T21:00:00Z,https://img-s-msn-com.akamaized.net/tenant/amp...,Even if you're not one of the 4.5 million Amer...,N9580,finance,finance-insurance,The States Where Dogs Put the Biggest Bite on ...,NaN,[],[]
21,https://assets.msn.com/labs/mind/AADm4XB.html,AADm4XB,The Biggest Mental and Physical Benefits of Wo...,2018-06-25T19:42:03Z,https://img-s-msn-com.akamaized.net/tenant/amp...,We've got some happy news that will rev up you...,N22632,health,fitness,The Biggest Mental and Physical Benefits of Wo...,Exercise does your mind and body good almost i...,"[{""Label"": ""Exercise"", ""Type"": ""C"", ""WikidataI...",[]
22,https://assets.msn.com/labs/mind/AAJwp2v.html,AAJwp2v,A Place to Call Home: These adventurous siblin...,2019-10-29T13:52:16Z,NaN,"Adventurous, fun and loving describe 13-year-o...",N16951,news,newsus,A Place to Call Home: These adventurous siblin...,"Adventurous, fun and loving describe 13-year-o...",[],[]


In [46]:
tr_nids = []
for nid in demo_tr.iloc[:,0].tolist():
    if nid not in demo_trains.iloc[:,6].tolist():
        tr_nids.append(nid)

print(len(tr_nids))

not_tr = demo_tr.loc[demo_tr.iloc[:,0].isin(tr_nids)]
not_tr.head()

5733


,0,1,2,3,4,5,6,7
1,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
7,N9552,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,https://www.msn.com/en-us/health/nutrition/50-...,[],[]
11,N15322,health,health-news,"Michigan apple recall: Nearly 2,300 crates cou...",A Michigan produce company has recalled nearly...,https://www.msn.com/en-us/health/health-news/m...,"[{""Label"": ""Michigan"", ""Type"": ""G"", ""WikidataI...","[{""Label"": ""Michigan"", ""Type"": ""G"", ""WikidataI..."
17,N25192,lifestyle,lifestyleroyals,Every outfit Duchess Kate has worn in 2019,See Kate Middleton's style choices this year f...,https://www.msn.com/en-us/lifestyle/lifestyler...,[],"[{""Label"": ""Catherine, Duchess of Cambridge"", ..."
18,N25099,health,weightloss,These Simple Diet Changes Helped This Guy Lose...,The part of me that knew I had to make changes...,https://www.msn.com/en-us/health/weightloss/th...,[],[]


# 크롤링

In [6]:
url_lst = []
titles = []    
# dates = []
contents = []
images = []

urls = not_tr.iloc[:,5].tolist()

print('crawling start.')
start_time = int(time.time())

url_except =[]
num = 0
total = len(urls)
for url in urls:
    # 진행상황 확인 
    num += 1
    if num % 2000 == 0:
        print('{} urls are crawled.\n {} are remained'.format(num, total - num))

    try:
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')

        # url
        url_lst.append(url)

        # title
        if soup.find('h1').text.strip() is not None:
            title = soup.find('h1').text.strip()
            titles.append(title)
        elif soup.find('header').text.strip() is not None:
            title = soup.find('header').text.strip()
            titles.append(title)
        else:
            titles.append(None)

#         # date
#         try:
#             date = soup.find('time').get('datetime')
#             dates.append(date)
#         except:
#             dates.append(None)

        # content
        if soup.find_all('p') is not None:
            content = []
            for el in soup.find_all('p'):
                content.append(el.get_text()) 
            contents.append(' '.join(content))
        elif soup.find('h2').text.strip() is not None:
            content = soup.find('h2').text.strip()
            contents.append(content)
        else:
            contents.append(url)

        # image
        try:
            image = soup.find('img').get('src')
            images.append(image)
        except:
            images.append('no image')



        # 길이 맞추기 
        if len(url_lst) != num:
            url_lst.append(None)
        if len(titles) != num:
            titles.append(None)
#         if len(dates) != num:
#             dates.append(None)
        if len(contents) != num:
            contents.append(None)
        if len(images) != num:
            images.append(None)

    except:
        url_except.append(url)

print('-------Seconds: %s--------' % (time.time() - start_time))

crawling start.
2000 urls are crawled.
 3733 are remained
4000 urls are crawled.
 1733 are remained
-------Seconds: 1851.7829959392548--------


In [7]:
print('len url_lst', len(url_lst))
print('len titles', len(titles))
# print('len dates', len(dates))
print('len contents', len(contents))
print('len images', len(images))

len url_lst 5733
len titles 5733
len dates 5733
len contents 5733
len images 5733


# date 수집

In [83]:
print('추가 수집할 길이',len(urls))

추가 수집할 길이 5733


In [84]:
from tqdm import tqdm
from selenium import webdriver
import time

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument('User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36')

chromedriver = '/Users/baeyuna/Documents/chromedriver'
driver = webdriver.Chrome(chromedriver, options=options) # webdriver.ChromeOptions()

In [86]:
import time
from random import *

MAX_SLEEP_TIME = 5
urls = not_tr.iloc[:,5].tolist()
dates = []

for url in tqdm(urls[:500], desc="tqdm url going"):
    rand_value = randint(1, MAX_SLEEP_TIME)
    time.sleep(rand_value)

    driver.get(url)
    try:
        date = driver.find_element_by_xpath('//*[@id="precontent"]/section/div[1]/div/div/span[3]/time')
        dates.append(date)
    except:
        dates.append(None)
    
print(len(dates))




tqdm url going:   0%|          | 0/500 [00:04<?, ?it/s]


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59442): Max retries exceeded with url: /session/744433b14dcc759149b8b550410a875c/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f89e0051710>: Failed to establish a new connection: [Errno 61] Connection refused'))

### ip 우회하기

In [125]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument('User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36')

PROXY = "41.216.230.148:44795" # IP:Port

webdriver.DesiredCapabilities.CHROME['proxy'] = {
    "httpProxy": PROXY,
    "ftpProxy": PROXY,
    "sslProxy": PROXY,
    "proxyType": "MANUAL"
}

chromedriver = '/Users/baeyuna/Documents/chromedriver'
driver = webdriver.Chrome(chromedriver, options=options)

In [130]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument('User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36')
options.add_argument('lang=ko_KR')

# /Users/baeyuna/Documents/Crawling/xpath.ipynb
chromedriver = '/Users/baeyuna/Documents/chromedriver'
driver = webdriver.Chrome(chromedriver, options=options)

In [143]:
not_tr.iloc[:,5].tolist()

['https://www.msn.com/en-us/news/world/the-cost-of-trumps-aid-freeze-in-the-trenches-of-ukraines-war/ar-AAJgNsz?ocid=chopendata',
 'https://www.msn.com/en-us/health/nutrition/50-foods-you-should-never-eat,-according-to-health-experts/ss-AABDHTv?ocid=chopendata',
 'https://www.msn.com/en-us/health/health-news/michigan-apple-recall-nearly-2-300-crates-could-be-contaminated-with-listeria/ar-AAJwfO8?ocid=chopendata',
 'https://www.msn.com/en-us/lifestyle/lifestyleroyals/every-outfit-duchess-kate-has-worn-in-2019/ss-AABvaDh?ocid=chopendata',
 'https://www.msn.com/en-us/health/weightloss/these-simple-diet-changes-helped-this-guy-lose-75-pounds-in-9-months/ar-AAB207a?ocid=chopendata',
 'https://www.msn.com/en-us/sports/basketball_nba/la-rivalry-takes-on-new-meaning-for-fans-as-lakers,-clippers-in-hunt-for-nba-championship/ar-AAJaBQj?ocid=chopendata',
 'https://www.msn.com/en-us/sports/golf/randalls-rant-is-tiger-already-the-goat-without-18-majors/ar-AAJwfU7?ocid=chopendata',
 'https://www.msn

In [144]:
df = pd.DataFrame({'urls':not_tr.iloc[:,5].tolist()})
df.to_csv('not_tr.csv', index=False)

In [126]:
import time
from random import *
from tqdm import notebook

MAX_SLEEP_TIME = 5
urls = not_tr.iloc[:,5].tolist()
dates = []
except_urls = []

num = 0
for url in notebook.tqdm(urls[:5], desc="tqdm url going"):
    num += 1
    rand_value = randint(1, MAX_SLEEP_TIME)
    time.sleep(rand_value)
    
    try:
        driver.get(url)
        
        # date
        try: 
            date = driver.find_element_by_xpath('//*[@id="precontent"]/section/div[1]/div/div/span[3]/time')
            d = date.get_attribute('datetime')
            print(d)
            dates.append(d)
        except:
            dates.append(None)
    except:
        except_urls.append(url)
        
    if len(dates) != num:ㅁ
        dates.append(None)
    
print(len(dates))


5


In [133]:
urls[2]

'https://www.msn.com/en-us/health/health-news/michigan-apple-recall-nearly-2-300-crates-could-be-contaminated-with-listeria/ar-AAJwfO8?ocid=chopendata'

In [139]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument('User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36')
options.add_argument('lang=ko_KR')

# /Users/baeyuna/Documents/Crawling/xpath.ipynb
chromedriver = '/Users/baeyuna/Documents/chromedriver'
driver = webdriver.Chrome(chromedriver, options=options) # webdriver.ChromeOptions()

In [140]:
urls = 'https://www.msn.com/en-us/health/health-news/michigan-apple-recall-nearly-2-300-crates-could-be-contaminated-with-listeria/ar-AAJwfO8?ocid=chopendata'
urls

'https://www.msn.com/en-us/health/health-news/michigan-apple-recall-nearly-2-300-crates-could-be-contaminated-with-listeria/ar-AAJwfO8?ocid=chopendata'

In [141]:
driver.get(urls)

In [142]:
title = driver.find_element_by_xpath('//*[@id="precontent"]/section/div[1]/div/div/span[3]/time')

print(title.get_attribute('datetime'))

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="precontent"]/section/div[1]/div/div/span[3]/time"}
  (Session info: headless chrome=84.0.4147.105)


In [134]:
driver.get(urls[2])
date = driver.find_element_by_xpath('//*[@id="precontent"]/section/div[1]/div/div/span[3]/time')
date.get_attribute('datetime')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="precontent"]/section/div[1]/div/div/span[3]/time"}
  (Session info: headless chrome=84.0.4147.105)


In [127]:
date_df = pd.DataFrame({'date':date, 'url':urls[:5]})
print('잘 뽑힌 값 개수:',date_df.loc[~date_df['date'].isnull()].shape)
date_df

잘 뽑힌 값 개수: (5, 2)


,date,url
0,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/news/world/the-cost-...
1,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/health/nutrition/50-...
2,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/health/health-news/m...
3,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/lifestyle/lifestyler...
4,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/health/weightloss/th...


In [128]:
date_df['date'][2].get_attribute('datetime')

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=51096): Max retries exceeded with url: /session/31a4ef6ff3e0900ba97eb36bb7c6a169/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f89e0031b50>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
import time
from random import *
from tqdm import notebook

MAX_SLEEP_TIME = 5
urls = not_tr.iloc[:,5].tolist()
dates = []
except_urls = []

num = 0
for url in notebook.tqdm(urls[:3000], desc="tqdm url going"):
    num += 1
    rand_value = randint(1, MAX_SLEEP_TIME)
    time.sleep(rand_value)
    
    try:
        driver.get(url)
        
        # date
        try: 
            date = driver.find_element_by_xpath('//*[@id="precontent"]/section/div[1]/div/div/span[3]/time')
            dates.append(date.get_attribute('datetime'))
        except:
            dates.append(None)
    except:
        except_urls.append(url)
        
    if len(dates) != num:
        dates.append(None)
    
print(len(dates))


In [ ]:
date_df = pd.DataFrame({'date':date, 'url':urls[:3000]})
print('잘 뽑힌 값 개수:',date_df.loc[~date_df['date'].isnull()].shape)
date_df

In [ ]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument('User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36')

PROXY = "81.211.95.54:8080" # IP:Port

webdriver.DesiredCapabilities.CHROME['proxy'] = {
    "httpProxy": PROXY,
    "ftpProxy": PROXY,
    "sslProxy": PROXY,
    "proxyType": "MANUAL"
}

chromedriver = '/Users/baeyuna/Documents/chromedriver'
driver = webdriver.Chrome(chromedriver)

In [ ]:
import time
from random import *
from tqdm import notebook

MAX_SLEEP_TIME = 5
urls = not_tr.iloc[:,5].tolist()
dates = []
except_urls = []

num = 0
for url in notebook.tqdm(urls[3000:], desc="tqdm url going"):
    num += 1
    rand_value = randint(1, MAX_SLEEP_TIME)
    time.sleep(rand_value)
    
    try:
        driver.get(url)
        
        # date
        try: 
            date = driver.find_element_by_xpath('//*[@id="precontent"]/section/div[1]/div/div/span[3]/time')
            dates.append(date.get_attribute('datetime'))
        except:
            dates.append(None)
    except:
        except_urls.append(url)
        
    if len(dates) != num:
        dates.append(None)
    
print(len(dates))


In [ ]:
date_df = pd.DataFrame({'date':date, 'url':urls})
print('잘 뽑힌 값 개수:',date_df.loc[~date_df['date'].isnull()].shape)
date_df

In [ ]:
demo = pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/demo_tr_news.csv')
demo.head()

In [37]:
df = pd.DataFrame({'url' : url_lst,
                   'Title':titles,
                   'Date': dates, 
                   'Content': contents,
                   'Image': images})

# 데이터 저장   
print('-------df--------')
print(df.shape)
df

-------df--------
(5733, 5)


,url,Title,Date,Content,Image
0,https://www.msn.com/en-us/news/world/the-cost-...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
1,https://www.msn.com/en-us/health/nutrition/50-...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
2,https://www.msn.com/en-us/health/health-news/m...,"Michigan apple recall: Nearly 2,300 crates cou...",None,\n\n\n\n\n\n© North Bay Produce\n Michigan's N...,//static-entertainment-eas-s-msn-com.akamaized...
3,https://www.msn.com/en-us/lifestyle/lifestyler...,Every outfit Duchess Kate has worn in 2019,None,Duchess Kate rarely puts a foot wrong when it ...,//static-entertainment-eas-s-msn-com.akamaized...
4,https://www.msn.com/en-us/health/weightloss/th...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
...,...,...,...,...,...
5728,https://www.msn.com/en-us/news/elections-2020-...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
5729,https://www.msn.com/en-us/news/newsus/man-kill...,Man killed in single-car crash after having me...,None,A 52-year-old man was killed after he had a me...,//static-entertainment-eas-s-msn-com.akamaized...
5730,https://www.msn.com/en-us/sports/football_nfl/...,Believe in 'Minkah Magic' as Fitzpatrick spark...,None,"\n\n\n\n\n\n© Charles LeClaire, USA TODAY Spor...",//static-entertainment-eas-s-msn-com.akamaized...
5731,https://www.msn.com/en-us/video/lifestyle/stil...,Young Northeast Florida fans flock to U.S. wom...,None,When the U.S. women's national soccer team arr...,//static-entertainment-eas-s-msn-com.akamaized...


In [8]:
df = pd.DataFrame({'url' : url_lst,
                   'Title':titles,
                   'Date': dates, 
                   'Content': contents,
                   'Image': images})

# 데이터 저장   
print('-------df--------')
print(df.shape)
df

-------df--------
(5733, 5)


,url,Title,Date,Content,Image
0,https://www.msn.com/en-us/news/world/the-cost-...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
1,https://www.msn.com/en-us/health/nutrition/50-...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
2,https://www.msn.com/en-us/health/health-news/m...,"Michigan apple recall: Nearly 2,300 crates cou...",None,\n\n\n\n\n\n© North Bay Produce\n Michigan's N...,//static-entertainment-eas-s-msn-com.akamaized...
3,https://www.msn.com/en-us/lifestyle/lifestyler...,Every outfit Duchess Kate has worn in 2019,None,Duchess Kate rarely puts a foot wrong when it ...,//static-entertainment-eas-s-msn-com.akamaized...
4,https://www.msn.com/en-us/health/weightloss/th...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
...,...,...,...,...,...
5728,https://www.msn.com/en-us/news/elections-2020-...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
5729,https://www.msn.com/en-us/news/newsus/man-kill...,Man killed in single-car crash after having me...,None,A 52-year-old man was killed after he had a me...,//static-entertainment-eas-s-msn-com.akamaized...
5730,https://www.msn.com/en-us/sports/football_nfl/...,Believe in 'Minkah Magic' as Fitzpatrick spark...,None,"\n\n\n\n\n\n© Charles LeClaire, USA TODAY Spor...",//static-entertainment-eas-s-msn-com.akamaized...
5731,https://www.msn.com/en-us/video/lifestyle/stil...,Young Northeast Florida fans flock to U.S. wom...,None,When the U.S. women's national soccer team arr...,//static-entertainment-eas-s-msn-com.akamaized...


In [9]:
df.loc[df['Title']=='Whoops!']

,url,Title,Date,Content,Image
0,https://www.msn.com/en-us/news/world/the-cost-...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
1,https://www.msn.com/en-us/health/nutrition/50-...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
4,https://www.msn.com/en-us/health/weightloss/th...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
8,https://www.msn.com/en-us/lifestyle/lifestyleb...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
9,https://www.msn.com/en-us/lifestyle/shop-appar...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
...,...,...,...,...,...
5697,https://www.msn.com/en-us/news/newspolitics/re...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
5701,https://www.msn.com/en-us/news/elections-2020-...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
5723,https://www.msn.com/en-us/sports/racing/nascar...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
5726,https://www.msn.com/en-us/finance/finance-comp...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...


In [15]:
import numpy as np

for i in range(len(df)):
    if df.loc[i, 'Title'] == 'Whoops!':
        df.loc[i, 'Title'] = np.nan 
        df.loc[i, 'Date'] = np.nan 
        df.loc[i, 'Content'] = np.nan 
        df.loc[i, 'Image'] = np.nan 

df.head()

,url,Title,Date,Content,Image
0,https://www.msn.com/en-us/news/world/the-cost-...,NaN,NaN,NaN,NaN
1,https://www.msn.com/en-us/health/nutrition/50-...,NaN,NaN,NaN,NaN
2,https://www.msn.com/en-us/health/health-news/m...,"Michigan apple recall: Nearly 2,300 crates cou...",None,\n\n\n\n\n\n© North Bay Produce\n Michigan's N...,//static-entertainment-eas-s-msn-com.akamaized...
3,https://www.msn.com/en-us/lifestyle/lifestyler...,Every outfit Duchess Kate has worn in 2019,None,Duchess Kate rarely puts a foot wrong when it ...,//static-entertainment-eas-s-msn-com.akamaized...
4,https://www.msn.com/en-us/health/weightloss/th...,NaN,NaN,NaN,NaN


In [11]:
print('-------url_except-----')
print(len(url_except))

-------url_except-----
133


In [17]:
print(demo_trains.shape)
print(df.shape)
print(pd.concat([demo_trains,df]).shape)

(21007, 13)
(5733, 5)
(26740, 14)


In [18]:
df.rename(columns = {'url' : 5}, inplace = True)
df.head()

,5,Title,Date,Content,Image
0,https://www.msn.com/en-us/news/world/the-cost-...,NaN,NaN,NaN,NaN
1,https://www.msn.com/en-us/health/nutrition/50-...,NaN,NaN,NaN,NaN
2,https://www.msn.com/en-us/health/health-news/m...,"Michigan apple recall: Nearly 2,300 crates cou...",None,\n\n\n\n\n\n© North Bay Produce\n Michigan's N...,//static-entertainment-eas-s-msn-com.akamaized...
3,https://www.msn.com/en-us/lifestyle/lifestyler...,Every outfit Duchess Kate has worn in 2019,None,Duchess Kate rarely puts a foot wrong when it ...,//static-entertainment-eas-s-msn-com.akamaized...
4,https://www.msn.com/en-us/health/weightloss/th...,NaN,NaN,NaN,NaN


In [19]:
demo_tr = pd.merge(not_tr, df, how='inner',on= 5)
print(demo_tr.shape)
demo_tr.head()

(5731, 12)


,0,1,2,3,4,5,6,7,Title,Date,Content,Image
0,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...",NaN,NaN,NaN,NaN
1,N9552,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,https://www.msn.com/en-us/health/nutrition/50-...,[],[],NaN,NaN,NaN,NaN
2,N15322,health,health-news,"Michigan apple recall: Nearly 2,300 crates cou...",A Michigan produce company has recalled nearly...,https://www.msn.com/en-us/health/health-news/m...,"[{""Label"": ""Michigan"", ""Type"": ""G"", ""WikidataI...","[{""Label"": ""Michigan"", ""Type"": ""G"", ""WikidataI...","Michigan apple recall: Nearly 2,300 crates cou...",None,\n\n\n\n\n\n© North Bay Produce\n Michigan's N...,//static-entertainment-eas-s-msn-com.akamaized...
3,N25192,lifestyle,lifestyleroyals,Every outfit Duchess Kate has worn in 2019,See Kate Middleton's style choices this year f...,https://www.msn.com/en-us/lifestyle/lifestyler...,[],"[{""Label"": ""Catherine, Duchess of Cambridge"", ...",Every outfit Duchess Kate has worn in 2019,None,Duchess Kate rarely puts a foot wrong when it ...,//static-entertainment-eas-s-msn-com.akamaized...
4,N25099,health,weightloss,These Simple Diet Changes Helped This Guy Lose...,The part of me that knew I had to make changes...,https://www.msn.com/en-us/health/weightloss/th...,[],[],NaN,NaN,NaN,NaN


In [22]:
demo_tr.columns.tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 'Title', 'Date', 'Content', 'Image']

In [23]:
demo_tr = demo_tr.iloc[:,[5,8,9,11,10,0,1,2,3,4,6,7]]
demo_tr.columns.tolist()

[5, 'Title', 'Date', 'Image', 'Content', 0, 1, 2, 3, 4, 6, 7]

In [24]:
demo_tr.rename(columns = {5 : '5'}, inplace = True)
demo_tr.rename(columns = {0 : '0'}, inplace = True)
demo_tr.rename(columns = {1 : '1'}, inplace = True)
demo_tr.rename(columns = {2 : '2'}, inplace = True)
demo_tr.rename(columns = {3 : '3'}, inplace = True)
demo_tr.rename(columns = {4 : '4'}, inplace = True)
demo_tr.rename(columns = {6 : '6'}, inplace = True)
demo_tr.rename(columns = {7 : '7'}, inplace = True)

In [28]:
demo_trains = demo_trains.drop(['nids'], axis=1)
demo_trains.columns.tolist()

['5', 'Title', 'Date', 'Image', 'Content', '0', '1', '2', '3', '4', '6', '7']

In [29]:
demo = pd.concat([demo_tr, demo_trains])
print(demo.shape)
demo.head()

(26738, 12)


,5,Title,Date,Image,Content,0,1,2,3,4,6,7
0,https://www.msn.com/en-us/news/world/the-cost-...,NaN,NaN,NaN,NaN,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
1,https://www.msn.com/en-us/health/nutrition/50-...,NaN,NaN,NaN,NaN,N9552,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,[],[]
2,https://www.msn.com/en-us/health/health-news/m...,"Michigan apple recall: Nearly 2,300 crates cou...",None,//static-entertainment-eas-s-msn-com.akamaized...,\n\n\n\n\n\n© North Bay Produce\n Michigan's N...,N15322,health,health-news,"Michigan apple recall: Nearly 2,300 crates cou...",A Michigan produce company has recalled nearly...,"[{""Label"": ""Michigan"", ""Type"": ""G"", ""WikidataI...","[{""Label"": ""Michigan"", ""Type"": ""G"", ""WikidataI..."
3,https://www.msn.com/en-us/lifestyle/lifestyler...,Every outfit Duchess Kate has worn in 2019,None,//static-entertainment-eas-s-msn-com.akamaized...,Duchess Kate rarely puts a foot wrong when it ...,N25192,lifestyle,lifestyleroyals,Every outfit Duchess Kate has worn in 2019,See Kate Middleton's style choices this year f...,[],"[{""Label"": ""Catherine, Duchess of Cambridge"", ..."
4,https://www.msn.com/en-us/health/weightloss/th...,NaN,NaN,NaN,NaN,N25099,health,weightloss,These Simple Diet Changes Helped This Guy Lose...,The part of me that knew I had to make changes...,[],[]


In [30]:
demo.to_csv('demo_tr_news.csv', index=False)